# Multi-Step Financial & Search Agent

This notebook implements a multi-step agent using **LiquidAI/LFM2.5-1.2B-Instruct**.
The agent is designed to:
1. Perform arithmetic (+, -, *, /).
2. Fetch latest crypto and stock prices via `yfinance`.
3. Search for latest news via `Tavily`.

### Agent Flow:
1. **Router**: Analyzes the query and identifies necessary tool calls (JSON output).
2. **Executor**: Executes identified tool calls and retrieves results.
3. **Synthesizer**: Combines the results into a final cohesive answer.

## 1. Install Dependencies

In [1]:
!pip install -q transformers torch yfinance tavily-python accelerate

## 2. Configuration & Model Loading

### 2.1 API Keys

In [2]:
import os
from getpass import getpass

# Set API Keys
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
except:
    HF_TOKEN = os.getenv('HF_TOKEN')
    TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

if not HF_TOKEN:
    HF_TOKEN = getpass("Enter your Hugging Face Token (HF_TOKEN): ")

if not TAVILY_API_KEY:
    TAVILY_API_KEY = getpass("Enter your Tavily API Key (TAVILY_API_KEY): ")

# Export to environment for tools to use if needed
os.environ['HF_TOKEN'] = HF_TOKEN
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

### 2.2 Model Loading

In [3]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

if torch.cuda.is_available():
    try:
        major, minor = torch.cuda.get_device_capability()
        DEVICE = "cuda" if major >= 7 else "cpu"
    except: DEVICE = "cpu"
else: DEVICE = "cpu"

TORCH_DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
MODEL_NAME = "LiquidAI/LFM2.5-1.2B-Instruct"

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=TORCH_DTYPE, device_map="auto" if DEVICE == "cuda" else None, token=HF_TOKEN)
if DEVICE == "cpu": model = model.to("cpu")

def generate(messages, max_new_tokens=512):
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=False)

Loading LiquidAI/LFM2.5-1.2B-Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## 3. Tool Definitions

In [4]:
import yfinance as yf
import re, ast, json
from tavily import TavilyClient

def arithmetic_tool(op, a, b):
    try:
        a, b = float(a), float(b)
        if op == 'add': return a + b
        if op == 'subtract': return a - b
        if op == 'multiply': return a * b
        if op == 'divide': return a / b if b != 0 else "Error: Div0"
    except: return "Math Error"
    return "Invalid Op"

def price_tool(symbol):
    try:
        if symbol.upper() in ["BTC", "ETH", "SOL"]: symbol = f"{symbol.upper()}-USD"
        t = yf.Ticker(symbol)
        h = t.history(period="1d")
        return float(h['Close'].iloc[-1]) if not h.empty else "No price"
    except: return "Price Error"

def news_tool(query):
    if not TAVILY_API_KEY: return "No API Key"
    try:
        c = TavilyClient(api_key=TAVILY_API_KEY)
        r = c.search(query=query, search_depth="basic")
        return "\n".join([f"- {res['title']}" for res in r.get('results', [])[:3]])
    except: return "Search Error"

TOOLS_MAP = {"arithmetic": arithmetic_tool, "get_price": price_tool, "get_news": news_tool}
TOOLS_SCHEMA = [{"name": "arithmetic", "description": "Math", "parameters": {"type": "object", "properties": {"op": {"type": "string"}, "a": {"type": "number"}, "b": {"type": "number"}}, "required": ["op", "a", "b"]}}, {"name": "get_price", "description": "Price", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}, {"name": "get_news", "description": "News", "parameters": {"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]}}]

## 4. Agent Implementation

In [5]:
def parse_tool_calls(response_text):
    if not response_text: return []
    # Robust pattern for <|tool_call_start|>[name(args)]<|tool_call_end|>
    pattern = r'<\|tool_call_start\|>\[(\w+)\((.*?)\)\]<\|tool_call_end\|>'
    matches = re.finditer(pattern, response_text)
    calls = []
    for match in matches:
        try:
            name = match.group(1)
            args_raw = match.group(2)
            args_str = args_raw if args_raw is not None else ""
            args = {}
            if args_str.strip():
                # Split by comma respecting quotes
                parts = re.split(r',(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)', args_str)
                for part in parts:
                    if '=' in part:
                        k, v = part.split('=', 1)
                        try: args[k.strip()] = ast.literal_eval(v.strip())
                        except: args[k.strip()] = v.strip().strip('\"').strip("'")
            calls.append({"name": name, "arguments": args})
        except: continue
    return calls

class MultiStepAgent:
    def __init__(self, schema, tmap):
        self.schema, self.tmap = schema, tmap
        self.sys = f"You are helpful. Tools: {json.dumps(self.schema)}. Use <|tool_call_start|>[name(args)]<|tool_call_end|>."
    def run(self, query, max_iter=5):
        msgs = [{"role": "system", "content": self.sys}, {"role": "user", "content": query}]
        for i in range(max_iter):
            print(f"[*] Step {i+1}...")
            res = generate(msgs)
            calls = parse_tool_calls(res)
            if not calls: return re.sub(r'<\|.*?\|>', '', res).strip()
            msgs.append({"role": "assistant", "content": res})
            results = []
            for c in calls:
                n, a = c["name"], c["arguments"]
                print(f"[*] Call: {n}({a})")
                if n in self.tmap:
                    try: results.append(self.tmap[n](**a))
                    except Exception as e: results.append(f"Error: {e}")
                else: results.append(f"Tool {n} missing")
            msgs.append({"role": "tool", "content": json.dumps(results)})
        return "Max steps reached."

## 5. Testing the Agent

In [7]:
agent = MultiStepAgent(TOOLS_SCHEMA, TOOLS_MAP)
queries = [
    "Calculate 1024 times 4 and give me the latest Bitcoin price.",
    "What is the latest news for Nvidia and what is their current stock price?",
    "If I take the current Ethereum price and add 500, what is the result?"
]
for q in queries:
    print("="*30)
    print(f"USER: {q}")
    print(f"AGENT: {agent.run(q)}")

USER: Calculate 1024 times 4 and give me the latest Bitcoin price.
[*] Step 1...
[*] Call: arithmetic({'op': 'multiplication', 'a': 1024, 'b': '4)', 'get_price(symbol': 'BTC'})
[*] Step 2...
[*] Call: get_price({'symbol': 'BTC'})
[*] Step 3...
AGENT: The result of 1024 multiplied by 4 is 4096.  
The latest Bitcoin (BTC) price is approximately $87,891.66.
USER: What is the latest news for Nvidia and what is their current stock price?
[*] Step 1...
[*] Call: get_news({'query': 'Nvidia latest news")', 'get_price(symbol': 'NVDA'})
[*] Step 2...
[*] Call: get_price({'symbol': 'NVDA'})
[*] Step 3...
AGENT: The latest news for Nvidia is not available in my current data. However, the current stock price for Nvidia (NVDA) is $186.47. Let me know if you need further assistance!
USER: If I take the current Ethereum price and add 500, what is the result?
[*] Step 1...
[*] Call: get_price({'symbol': 'ETH'})
[*] Step 2...
[*] Call: arithmetic({'op': 'add', 'a': 2909.95703125, 'b': 500})
[*] Step 3..